In [4]:
import pandas as pd
import random

df = pd.read_csv('beers.csv')
ounce_to_ml = 29.574
sample_size = 300

beer_styles = df['style']
style_abundance = (beer_styles.value_counts(normalize=True) * 100).to_dict()


beers = df[['name', 'abv', 'ibu', 'ounces', 'style']].dropna()
beers['abv'] = round(100 * beers['abv'], 2)
beers['ounces'] *= ounce_to_ml
beers['ounces'] = beers['ounces'].astype('int')
beers['style'] = (beers['style'].apply(lambda x: style_abundance[x])).astype('float')
beers.columns = ['nazwa', 'procenty', 'ibu', 'ilosc ml', 'pospolitosc']
beers.reset_index(drop=True, inplace=True)
beers_sample = beers.sample(sample_size).reset_index(drop=True)
beers_sample['cena'] = [round(random.uniform(8, 20), 2) for _ in range(sample_size)]
beers_sample.head(10)

,nazwa,procenty,ibu,ilosc ml,pospolitosc,cena
0,Farmer Wirtz India Pale Ale,7.0,94.0,473,0.540541,17.52
1,Mexican Logger,4.2,18.0,354,1.621622,19.90
2,Snow King Pale Ale,6.0,55.0,354,10.187110,15.80
3,Point Special Lager,4.7,9.0,473,0.748441,14.99
4,Tonganoxie Honey Wheat,4.4,22.0,354,4.033264,17.71
5,ESB Special Ale,5.7,58.0,354,0.831601,18.15
6,Odyssey Imperial IPA,8.4,90.0,354,4.365904,9.05
7,Lobo Lito,4.0,12.0,354,0.498960,10.17
8,Boont Amber Ale (2011),5.8,15.0,354,5.530146,14.51
9,Day Hike Session,4.1,41.0,354,17.629938,15.71


In [5]:
beers_sample.to_csv('piwa_kraftowe_5_kolumn.csv')

Wygenerowanie wersji 2, 3 i 4 parametrowych przez usunięcie kolejnych kolumn z wersji 5 parametrowej.

In [6]:
for i, column in enumerate(['pospolitosc', 'ibu', 'ilosc ml']):
    beers_sample.drop(labels=column, axis=1, inplace=True)
    beers_sample.to_csv(f'piwa_kraftowe_{4-i}_kolumn.csv')